In [1]:
import os
os.chdir("../")

In [2]:
%pwd

'/home/sidd/kidney-disease-classification'

In [25]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    params_learning_rate: float
    params_weights: str
    params_include_top: bool

In [26]:
from CNN_Classifier.constants import *
from CNN_Classifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [27]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir,"kidney-data", "CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone","CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone")
        create_directories([
            Path(training.root_dir)
        ])
        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE,
            params_learning_rate=params.LEARNING_RATE,
            params_weights=params.WEIGHTS,
            params_include_top=params.INCLUDE_TOP
        )
        return training_config

In [28]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [31]:
class Training:
    def __init__(self, config:TrainingConfig):
        self.config = config
        self.validation_ratio = 0.2


    def get_base_model(self):
        self.model = tf.keras.applications.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )
        self.model.trainable = False

        flatten_in = tf.keras.layers.Flatten()(self.model.output)
        prediction = tf.keras.layers.Dense(
            units=4,
            activation="softmax"
        )(flatten_in)

        self.full_model = tf.keras.Model(
            inputs=self.model.input,
            outputs=prediction
        )

        opt = tf.keras.optimizers.RMSprop(
            learning_rate=self.config.params_learning_rate,
            rho=0.9,
            momentum=0.0,
            epsilon=1e-07,
            centered=False,
            weight_decay=None,
            clipnorm=None,
            clipvalue=None,
            global_clipnorm=None,
            use_ema=False,
            ema_momentum=0.99,
            ema_overwrite_frequency=100,
            name="rmsprop",
        )

        self.full_model.compile(
            optimizer=opt,
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=['accuracy']
        )

        self.full_model.summary()
        


    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train_valid_generator(self):
        self.train_data = tf.keras.utils.image_dataset_from_directory(
            self.config.training_data,
            labels="inferred",
            label_mode="categorical",
            class_names=['Cyst', 'Normal', 'Stone', 'Tumor'],
            color_mode="rgb",
            batch_size=self.config.params_batch_size,
            image_size=self.config.params_image_size[:-1],
            shuffle=True,
            seed=69,
            validation_split=self.validation_ratio,
            subset="training",
            interpolation="bilinear",
            verbose=True,
        )


    def train(self):
        self.full_model.summary()
        self.full_model.fit(
            self.train_data,
            epochs=self.config.params_epochs,
            shuffle=True,
            batch_size=self.config.params_batch_size,
        )
        self.save_model(
            path=self.config.trained_model_path,
            model=self.full_model
        )

In [32]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()

except Exception as e:
    raise e

[2024-04-06 11:32:45,949: INFO: common: yaml file: config/config.yaml loaded successfully:]
[2024-04-06 11:32:45,952: INFO: common: yaml file: params.yaml loaded successfully:]
[2024-04-06 11:32:45,953: INFO: common: created directory at: artifacts:]
[2024-04-06 11:32:45,953: INFO: common: created directory at: artifacts/training:]


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │       100,356 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,815,044 (56.51 MB)

 Trainable params: 100,356 (392.02 KB)

...

 1/30 ━━━━━━━━━━━━━━━━━━━━ 2:48 6s/step - accuracy: 0.5000 - loss: 9.0311

2024-04-06 11:32:52.306219: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.
2024-04-06 11:32:52.567923: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.


 2/30 ━━━━━━━━━━━━━━━━━━━━ 2:12 5s/step - accuracy: 0.5391 - loss: 20.4135

2024-04-06 11:32:57.031796: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.


30/30 ━━━━━━━━━━━━━━━━━━━━ 143s 5s/step - accuracy: 0.7679 - loss: 14.3005
Epoch 2/5
30/30 ━━━━━━━━━━━━━━━━━━━━ 131s 4s/step - accuracy: 0.9738 - loss: 0.9350
Epoch 3/5
30/30 ━━━━━━━━━━━━━━━━━━━━ 137s 5s/step - accuracy: 0.9934 - loss: 0.1708
Epoch 4/5
30/30 ━━━━━━━━━━━━━━━━━━━━ 142s 5s/step - accuracy: 1.0000 - loss: 5.0964e-10
Epoch 5/5
30/30 ━━━━━━━━━━━━━━━━━━━━ 145s 5s/step - accuracy: 1.0000 - loss: 2.0279e-10
